In [2]:
# Remember: library imports are ALWAYS at the top of the script, no exceptions!
import sqlite3
import os
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.impute import KNNImputer
from ydata_profiling import ProfileReport

/Users/carlos.costa/anaconda3/envs/DM2324/lib/python3.11/site-packages/numba/core/decorators.py:262: NumbaDeprecationWarning: numba.generated_jit is deprecated. Please see the documentation at: https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-generated-jit for more information and advice on a suitable replacement.
  warnings.warn(msg, NumbaDeprecationWarning)
/Users/carlos.costa/anaconda3/envs/DM2324/lib/python3.11/site-packages/visions/backends/shared/nan_handling.py:50: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @nb.jit


# Context
The data we will be using through the pratical classes comes from a small relational database whose schema can be seen below:
![alt text](./schema.png "Relation database schema")

# Reading the Data

In [3]:
# path to database
my_path = os.path.join("./datamining.db")

# connect to the database
conn = sqlite3.connect(my_path)

# the query
query = """
select
    age, 
    income, 
    frq, 
    rcn, 
    mnt, 
    clothes, 
    kitchen, 
    small_appliances, 
    toys, 
    house_keeping,
    dependents, 
    per_net_purchase,
    g.gender, 
    e.education, 
    m.status, 
    r.description
from customers as c
    join genders as g on g.id = c.gender_id
    join education_levels as e on e.id = c.education_id
    join marital_status as m on m.id = c.marital_status_id
    join recommendations as r on r.id = c.recommendation_id
order by c.id;
"""

df = pd.read_sql_query(query, conn)

## Make a copy of your original dataset

why?

In [6]:
# CODE HERE
df_original = df.copy()

# Metadata
- *id* - The unique identifier of the customer
- *age* - The year of birht of the customer
- *income* - The income of the customer
- *frq* - Frequency: number of purchases made by the customer
- *rcn* - Recency: number of days since last customer purchase
- *mnt* - Monetary: amount of € spent by the customer in purchases
- *clothes* - Number of clothes items purchased by the customer
- *kitchen* - Number of kitchen items purchased by the customer
- *small_appliances* - Number of small_appliances items purchased by the customer
- *toys* - Number of toys items purchased by the customer
- *house_keeping* - Number of house_keeping items purchased by the customer
- *dependents* - Binary. Whether or not the customer has dependents
- *per_net_purchase* - Percentage of purchases made online
- *education* - Education level of the customer
- *status* - Marital status of the customer
- *gender* - Gender of the customer
- *description* - Last customer's recommendation description

## Problems:
- Duplicates?
- Data types?
- Missing values?
- Strange values?
- Descriptive statistics?

### Take a closer look and point out possible problems:

(hint: a missing values in pandas is represented with a NaN value)

In [7]:
# replace "" by nans
df.replace("", np.nan, inplace=True)

In [8]:
# check dataset data types again
df.dtypes

age                   int64
income              float64
frq                   int64
rcn                   int64
mnt                   int64
clothes               int64
kitchen               int64
small_appliances      int64
toys                  int64
house_keeping         int64
dependents          float64
per_net_purchase      int64
gender               object
education            object
status               object
description          object
dtype: object

In [9]:
# check descriptive statistics again
df.describe(include="all").T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
age,8998.0,NaN,NaN,NaN,1966.05968,17.296552,1936.0,1951.0,1966.0,1981.0,1996.0
income,8952.0,NaN,NaN,NaN,69963.550827,27591.556226,10000.0,47741.0,70030.5,92218.0,140628.0
frq,8998.0,NaN,NaN,NaN,19.848077,10.903435,3.0,10.0,17.0,28.0,59.0
rcn,8998.0,NaN,NaN,NaN,62.469771,69.761802,0.0,26.0,53.0,79.0,549.0
mnt,8998.0,NaN,NaN,NaN,622.162814,646.768205,6.0,63.0,383.0,1076.0,3052.0
clothes,8998.0,NaN,NaN,NaN,50.446655,23.422249,1.0,33.0,51.0,69.0,99.0
kitchen,8998.0,NaN,NaN,NaN,7.039675,7.848139,0.0,2.0,4.0,10.0,75.0
small_appliances,8998.0,NaN,NaN,NaN,28.524116,12.586437,1.0,19.0,28.0,37.0,74.0
toys,8998.0,NaN,NaN,NaN,7.036897,7.924422,0.0,2.0,4.0,10.0,62.0
house_keeping,8998.0,NaN,NaN,NaN,6.929984,7.882655,0.0,2.0,4.0,9.0,77.0


In [10]:
#Define metric and non-metric features. Why?
non_metric_features = ["education", "status", "gender", "dependents", "description"]
metric_features = df.columns.drop(non_metric_features).to_list()

## Fill missing values (Data imputation)

How can we fill missing values?

In [11]:
# Creating a copy to apply central tendency measures imputation
df_central = df.copy()

In [12]:
# count of missing values
df_central.isna().sum()

age                   0
income               46
frq                   0
rcn                   0
mnt                   0
clothes               0
kitchen               0
small_appliances      0
toys                  0
house_keeping         0
dependents          282
per_net_purchase      0
gender                0
education            47
status              177
description           0
dtype: int64

In [17]:
medians = df_central.median(numeric_only=True)

In [16]:
# Check each step of these commands
modes = df_central[non_metric_features].mode().loc[0]
modes

education      Graduation
status            Married
gender                  M
dependents            1.0
description      OK nice!
Name: 0, dtype: object

In [23]:
# Fill NaNs on df_central
# CODE HERE

# v1 
# df_central = df_central.fillna(medians).fillna(modes)

# v2
df_central.fillna(pd.concat([medians, modes]), inplace=True)

df_central.isna().sum()

age                 0
income              0
frq                 0
rcn                 0
mnt                 0
clothes             0
kitchen             0
small_appliances    0
toys                0
house_keeping       0
dependents          0
per_net_purchase    0
gender              0
education           0
status              0
description         0
dtype: int64

In [24]:
# Creating new df copy to explore neighbordhood imputation
df_neighbors = df.copy()

In [27]:
# Seeing rows with NaNs
nans_index = df_neighbors[metric_features].isna().any(axis=1)
df_neighbors[nans_index]

,age,income,frq,rcn,mnt,clothes,kitchen,small_appliances,toys,house_keeping,dependents,per_net_purchase,gender,education,status,description
61,1968,NaN,14,90,184,95,1,3,1,0,1.0,59,F,2nd Cycle,Married,Meh...
703,1988,NaN,10,77,39,24,5,63,4,4,1.0,68,F,2nd Cycle,Together,OK nice!
793,1956,NaN,15,25,338,72,1,25,0,1,1.0,48,M,Master,Single,OK nice!
852,1956,NaN,21,95,596,69,4,20,2,6,1.0,33,M,Graduation,Together,OK nice!
1197,1974,NaN,20,10,395,67,1,28,2,2,1.0,56,F,Graduation,Together,Meh...
1280,1949,NaN,8,81,122,76,3,18,2,1,1.0,38,M,Master,NaN,Meh...
1614,1978,NaN,7,55,51,70,4,20,1,5,1.0,41,M,Graduation,Married,OK nice!
1669,1987,NaN,21,5,482,39,3,43,8,7,1.0,36,F,PhD,Married,Kind of OK
1917,1951,NaN,19,24,593,49,10,30,10,1,1.0,23,M,Graduation,Married,Meh...
1924,1948,NaN,31,86,1468,46,3,40,6,5,0.0,11,F,Master,Divorced,OK nice!


In [30]:
# KNNImputer - only works for numerical variables. Fill NaNs on df_neighbors
imputer = KNNImputer(n_neighbors=5, weights="uniform")
# CODE HERE
df_neighbors[metric_features] = imputer.fit_transform(df_neighbors[metric_features])

In [31]:
# See rows with NaNs imputed
# CODE HERE
df_neighbors.isna().sum()

age                   0
income                0
frq                   0
rcn                   0
mnt                   0
clothes               0
kitchen               0
small_appliances      0
toys                  0
house_keeping         0
dependents          282
per_net_purchase      0
gender                0
education            47
status              177
description           0
dtype: int64

In [ ]:
# let's keep the central imputation
df = df_central.copy()

## An overview of our previous data exploration

You can also explore this dataset using the exported `pandas-profiling` report.




![](../figures/exp_analysis/categorical_variables_frequecies.png)

![](../figures/exp_analysis/numeric_variables_histograms.png)

![](../figures/exp_analysis/numeric_variables_boxplots.png)

![](../figures/exp_analysis/pairwise_relationship_of_numerical_variables.png)

![](../figures/exp_analysis/correlation_matrix.png)

## Outlier removal

Why do we need to remove outliers? Which methods can we use?


Let's start by "manually" filtering the dataset's outliers

In [38]:
# This may vary from session to session, and is prone to varying interpretations.
# A simple example is provided below:

filters1 = (
    (df['house_keeping'] < 50) &
    (df['toys'] < 50) &
    (df['kitchen'] < 50) &
    (df['education'] != 'OldSchool') &
    (df['status'] != 'Whatever')
)

df_1 = df[filters1]

In [39]:
print('Percentage of data kept after removing outliers:', np.round(df_1.shape[0] / df_original.shape[0], 4))

Percentage of data kept after removing outliers: 0.9912


### Outlier removal using only the IQR method

Why should you use/not use this method?

In [43]:
# Compute the IQR

# CODE HERE
# Calculate Q1, Q3, IQR
q25 = df[metric_features].quantile(0.25)
q75 = df[metric_features].quantile(0.75)
iqr = q75 - q25

# Compute upper and lower limit (lower_limit = Q1 -1.5*IQR | upper_limit = Q3 + 1.5*IQR)
upper_lim = q75 + 1.5 * iqr
lower_lim = q25 - 1.5 * iqr


filters2 = []
for metric in metric_features:
    llim = lower_lim[metric]
    ulim = upper_lim[metric]
    filters2.append(df[metric].between(llim, ulim, inclusive='both'))

df_2 = df[np.all(filters2, 0)]
print('Percentage of data kept after removing outliers:', np.round(df_2.shape[0] / df_original.shape[0], 4))

Percentage of data kept after removing outliers: 0.8195


## Combining different outlier methods

More robust/ consistent outlier detection method:

In [45]:
df_3 = df[(filters1 | np.all(filters2))]
print('Percentage of data kept after removing outliers:', np.round(df_3.shape[0] / df_original.shape[0], 4))

Percentage of data kept after removing outliers: 0.9912


In [46]:
# Get the manual filtering version
df = df_1.copy()

## Feature Engineering

A reminder of our metadata:
- *id* - The unique identifier of the customer
- *age* - The year of birht of the customer
- *income* - The income of the customer
- *frq* - Frequency: number of purchases made by the customer
- *rcn* - Recency: number of days since last customer purchase
- *mnt* - Monetary: amount of € spent by the customer in purchases
- *clothes* - Number of clothes items purchased by the customer
- *kitchen* - Number of kitchen items purchased by the customer
- *small_appliances* - Number of small_appliances items purchased by the customer
- *toys* - Number of toys items purchased by the customer
- *house_keeping* - Number of house_keeping items purchased by the customer
- *dependents* - Binary. Whether or not the customer has dependents
- *per_net_purchase* - Percentage of purchases made online
- *education* - Education level of the customer
- *status* - Marital status of the customer
- *gender* - Gender of the customer
- *description* - Last customer's recommendation description

In [49]:
# Try to pull as many features as you can; You can always drop them later on
# You can start by converting the 'age' variable to the actual age
# You can also store the birth year in another feature

# CODE HERE
df_ = df.copy()
df_.rename(columns={"age": "birth_year"}, inplace=True)

df_

,birth_year,income,frq,rcn,mnt,clothes,kitchen,small_appliances,toys,house_keeping,dependents,per_net_purchase,gender,education,status,description
0,1946,90782.0,33,66,1402,37,5,44,10,3,0.0,19,M,Graduation,Together,Take my money!!
1,1936,113023.0,32,6,1537,55,1,38,4,2,0.0,9,F,PhD,Divorced,Take my money!!
2,1990,28344.0,11,69,44,32,19,24,1,24,1.0,59,M,Graduation,Married,Kind of OK
3,1955,93571.0,26,10,888,60,10,19,6,5,1.0,35,F,Master,NaN,OK nice!
4,1955,91852.0,31,26,1138,59,5,28,4,4,1.0,34,F,Graduation,Together,Take my money!!
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8993,1960,94367.0,28,1,896,68,5,21,3,4,1.0,55,F,1st Cycle,Single,Take my money!!
8994,1975,58121.0,12,6,61,53,6,28,7,6,1.0,71,M,2nd Cycle,Single,Meh...
8995,1986,54292.0,29,72,1011,41,11,36,1,11,0.0,31,M,Graduation,Together,Take my money!!
8996,1938,125962.0,38,75,1668,61,2,25,5,6,1.0,45,M,2nd Cycle,Married,Take my money!!


## Variable transform
You may change the values of observations for a give variable:

In [ ]:
# Remember the 'rcn' variable had an odd behavior? 
# This is the time to fix that

# CODE HERE

## Variable selection: Redundancy VS Relevancy

### Redundancy: Handling highly correlated variables

We already saw our original correlation matrix:

![](../figures/exp_analysis/correlation_matrix.png)

In [ ]:
# Select variables according to their correlations

# CODE HERE
df.drop(..., inplace=True)


In [ ]:
# Updating metric_features

metric_features.append()
metric_features.remove()

metric_features


### Relevancy
Selecting variables based on the relevancy of each one to the task. Example: remove uncorrelated variables with the target, stepwise regression, use variables for product clustering, use variables for socio-demographic clustering, ...

Variables that aren't correlated with any other variable are often also not relevant. In this case we will not focus on this a lot since we don't have a defined task yet.

## Redo data exploration

Check if the data looks the way you expect it to. 

- Have you missed some outliers? 
- Are there still missing values?
- Is the data normalized?

This is an iterative process. It is likely you will change your preprocessing steps frequently throughout your group work.

In [ ]:
ProfileReport(
    df,
    title='Tugas Customer Data Preprocessed',
    correlations={
        "pearson": {"calculate": True},
        "spearman": {"calculate": False},
        "kendall": {"calculate": False},
        "phi_k": {"calculate": False},
        "cramers": {"calculate": False},
    },
)